In [1]:
pip install pandas lxml openpyxl


   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------  3.9/4.0 MB 26.0 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 20.2 MB/s  0:00:00


In [1]:
import re
import pandas as pd

# MOM public holidays page
MOM_URL = "https://www.mom.gov.sg/employment-practices/public-holidays"

# Helper functions

def clean_date_text(s: str) -> str:
    """
    Remove bracket notes and normalise spaces.
    Example:
      '29 January 2025 (Tuesday)' -> '29 January 2025'
    """
    s = str(s)
    s = re.sub(r"\(.*?\)", "", s)      # remove (in lieu), (observed), etc.
    s = re.sub(r"\s+", " ", s).strip()
    return s


def dmy(dt: pd.Timestamp) -> str:
    """
    Format date as D/M/YYYY (no leading zeros) to match your other files.
    Example: 1 January 2025 -> '1/1/2025'
    """
    return f"{dt.day}/{dt.month}/{dt.year}"


def fetch_tables():
    """Scrape all HTML tables from the MOM public holidays page."""
    return pd.read_html(MOM_URL)


def extract_year(year: int, tables):
    """
    Extract all public-holiday dates for a given year.

    Handles cases like:
      '29 January 2025 30 January 2025' → 29 Jan 2025 and 30 Jan 2025
    by using regex to find *all* 'D Month YYYY' patterns inside a cell.
    """
    out = []

    # pattern: e.g. '29 January 2025', '3 April 2026'
    date_pattern = re.compile(r"\d{1,2}\s+[A-Za-z]+\s+20\d{2}")

    for df in tables:
        # normalise column names
        df.columns = [str(c).strip().lower() for c in df.columns]

        if "date" not in df.columns:
            continue

        # holiday name column (if available)
        name_col = "public holiday" if "public holiday" in df.columns else None

        # keep rows where 'date' text mentions this year
        rows = df[df["date"].astype(str).str.contains(str(year))]
        if rows.empty:
            continue

        rows = rows.copy()
        rows["date_clean"] = rows["date"].astype(str).map(clean_date_text)

        for _, r in rows.iterrows():
            text = r["date_clean"]

            # find ALL date strings inside this cell (for CNY etc.)
            matches = date_pattern.findall(text)

            if matches:
                parts = matches
            else:
                # if nothing matched, fall back to entire cleaned text
                parts = [text]

            for part in parts:
                dt = pd.to_datetime(part, dayfirst=True, errors="coerce")
                if pd.isna(dt) or dt.year != year:
                    continue

                name = r[name_col] if name_col else "Public Holiday"
                out.append((dt.normalize(), str(name).strip()))

    # deduplicate & sort
    out = sorted(set(out), key=lambda x: x[0])
    return out


def build_calendar(year_start: int, year_end: int) -> pd.DataFrame:
    """
    Build a full daily calendar from 1 Jan year_start to 31 Dec year_end.

    Output columns:
      - date              (D/M/YYYY)
      - is_public_holiday ('TRUE' / 'FALSE')
      - day_of_week       (Monday, Tuesday, ...)
      - is_weekend        ('TRUE' for Sat/Sun, otherwise 'FALSE')
    """
    tables = fetch_tables()

    # collect all holiday dates across requested years
    holidays = []
    for y in range(year_start, year_end + 1):
        holidays.extend(extract_year(y, tables))

    holiday_dates = {d for d, _ in holidays}

    # build continuous date range
    start = pd.Timestamp(f"{year_start}-01-01")
    end = pd.Timestamp(f"{year_end}-12-31")
    dates = pd.date_range(start, end, freq="D")

    df = pd.DataFrame({
        "date": [dmy(d) for d in dates],
        "is_public_holiday": [
            "TRUE" if d.normalize() in holiday_dates else "FALSE"
            for d in dates
        ],
        "day_of_week": [d.day_name() for d in dates],
        "is_weekend": [
            "TRUE" if d.day_name() in ("Saturday", "Sunday") else "FALSE"
            for d in dates
        ],
    })

    return df

In [2]:
# Build the new years
df_2025_2026 = build_calendar(2025, 2026)
df_2025_2026
df_2025_2026.head(20)

,date,is_public_holiday,day_of_week,is_weekend
0,1/1/2025,TRUE,Wednesday,FALSE
1,2/1/2025,FALSE,Thursday,FALSE
2,3/1/2025,FALSE,Friday,FALSE
3,4/1/2025,FALSE,Saturday,TRUE
4,5/1/2025,FALSE,Sunday,TRUE
5,6/1/2025,FALSE,Monday,FALSE
6,7/1/2025,FALSE,Tuesday,FALSE
7,8/1/2025,FALSE,Wednesday,FALSE
8,9/1/2025,FALSE,Thursday,FALSE
9,10/1/2025,FALSE,Friday,FALSE


In [3]:
# Save CSV
df_2025_2026.to_csv("calendar_2025_2026.csv", index=False)